In [1]:
#Adaptive Efficient coordinate attention
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_attention(inputs, reduction_ratio=16, num_groups=8):
    # Efficient Channel Attention (ECA)
    channels = inputs.shape[-1]
    eca_avg = K.mean(inputs, axis=(1, 2))
    eca_attention = tf.keras.layers.Conv1D(1, kernel_size=1, activation='sigmoid')(eca_avg)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    x = inputs * eca_attention

    # Coordinate Attention (CA)
    ca_channel_avg = K.mean(x, axis=2, keepdims=True)
    ca_channel_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(ca_channel_avg)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    x = x * ca_channel_attention

    # Adaptive Attention
    adaptive_avg = K.mean(x, axis=(1, 2))
    adaptive_attention = tf.keras.layers.Dense(1, activation='sigmoid')(adaptive_avg)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    x = x * adaptive_attention

    return x

In [ ]:
#Adaptive Efficient coordinate attention
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_attention(inputs, reduction_ratio=16, num_groups=8):
    # Efficient Channel Attention (ECA)
    channels = inputs.shape[-1]
    eca_avg = K.mean(inputs, axis=(1, 2))
    eca_attention = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(eca_avg)
    eca_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(eca_attention)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    x = inputs * eca_attention

    # Coordinate Attention (CA)
    ca_channel_avg = K.mean(x, axis=2, keepdims=True)
    ca_channel_attention = tf.keras.layers.Conv1D(filters=channels // num_groups, kernel_size=1, activation='relu')(ca_channel_avg)
    ca_channel_attention = tf.keras.layers.Conv1D(filters=channels, kernel_size=1, activation='sigmoid')(ca_channel_attention)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    x = x * ca_channel_attention

    # Adaptive Attention
    adaptive_avg = K.mean(x, axis=(1, 2))
    adaptive_attention = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(adaptive_avg)
    adaptive_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(adaptive_attention)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    x = x * adaptive_attention

    return x

inputs = tf.keras.layers.Input((128, 128, 1))

# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
f1 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p1)
f1 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f1))
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
f2 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p2)
f2 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f2)


c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f2))
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
f3 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p3)
f3 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f3))
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
f4 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(p4)
f4 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(abs(f4))
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

# Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6_attended = custom_attention(u6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6_attended)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
f6 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v, tf.complex64)))(c6)
f6 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v, tf.complex64)))(f6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(abs(f6))
u7 = tf.keras.layers.concatenate([u7, c3])
u7_attended = custom_attention(u7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
f7 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v,tf.complex64)))(c7)
f7 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v,tf.complex64)))(f7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(abs(f7))
u8 = tf.keras.layers.concatenate([u8, c2])
u8_attended = custom_attention(u8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
f8 = tf.keras.layers.Lambda(lambda v: tf.signal.fft2d(tf.cast(v,tf.complex64)))(c8)
f8 = tf.keras.layers.Lambda(lambda v: tf.signal.ifft2d(tf.cast(v,tf.complex64)))(f8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(abs(f8))
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9_attended = custom_attention(u9)

c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_attention(inputs, reduction_ratio=16, num_groups=8):
    # Efficient Channel Attention (ECA)
    channels = inputs.shape[-1]
    eca_avg = K.mean(inputs, axis=(1, 2))
    eca_attention = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(eca_avg)
    eca_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(eca_attention)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    x = inputs * eca_attention

    # Coordinate Attention (CA)
    ca_channel_avg = K.mean(x, axis=2, keepdims=True)
    ca_channel_attention = tf.keras.layers.Conv1D(filters=channels // num_groups, kernel_size=1, activation='relu')(ca_channel_avg)
    ca_channel_attention = tf.keras.layers.Conv1D(filters=channels, kernel_size=1, activation='sigmoid')(ca_channel_attention)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    x = x * ca_channel_attention

    # Adaptive Attention
    adaptive_avg = K.mean(x, axis=(1, 2))
    adaptive_attention = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(adaptive_avg)
    adaptive_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(adaptive_attention)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    x = x * adaptive_attention

    return x

inputs = tf.keras.layers.Input((128, 128, 1))

# Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.2)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

# Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6_attended = custom_attention(u6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6_attended)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3], axis=3)
u7_attended = custom_attention(u7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7_attended)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2], axis=3)
u8_attended = custom_attention(u8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8_attended)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9_attended = custom_attention(u9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9_attended)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()



In [18]:
# coarse_net()
def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh((np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

def coarse_net():
    # Input layer
    inputs = tf.keras.layers.Input((128, 128, 1))

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='gelu', kernel_initializer='he_normal', padding='same')(c9)

    # Add another Conv2D layer with 16 filters and softmax activation
    outputs = tf.keras.layers.Conv2D(16, (1, 1), activation='softmax')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model

# Create the model
model = coarse_net()

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_162 (Conv2D)            (None, 128, 128, 16  160         ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 dropout_97 (Dropout)           (None, 128, 128, 16  0           ['conv2d_162[0][0]']             
                                )                                                             

In [19]:
from tensorflow.keras import backend as K

def custom_attention(inputs, reduction_ratio=16, num_groups=8):
    # Efficient Channel Attention (ECA)
    channels = inputs.shape[-1]
    eca_avg = K.mean(inputs, axis=(1, 2))
    eca_attention = tf.keras.layers.Conv1D(1, kernel_size=1, activation='sigmoid')(eca_avg)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    x = inputs * eca_attention

    # Coordinate Attention (CA)
    ca_channel_avg = K.mean(x, axis=2, keepdims=True)
    ca_channel_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(ca_channel_avg)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    x = x * ca_channel_attention

    # Adaptive Attention
    adaptive_avg = K.mean(x, axis=(1, 2))
    adaptive_attention = tf.keras.layers.Dense(1, activation='sigmoid')(adaptive_avg)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    x = x * adaptive_attention

    return x

def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh((np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

def coarse_net():
    # Input layer
    inputs = tf.keras.layers.Input((128, 128, 1))

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    u6 = custom_attention(u6)  # Apply custom attention
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    u7 = custom_attention(u7)  # Apply custom attention
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    u8 = custom_attention(u8)  # Apply custom attention
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(u8_attended)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    
    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    u9_attended = custom_attention(u9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9_attended)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    # Create the model
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_162 (Conv2D)            (None, 128, 128, 16  160         ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 dropout_97 (Dropout)           (None, 128, 128, 16  0           ['conv2d_162[0][0]']             
                                )                                                             

 conv2d_169 (Conv2D)            (None, 16, 16, 128)  147584      ['dropout_100[0][0]']            
                                                                                                  
 max_pooling2d_68 (MaxPooling2D  (None, 8, 8, 128)   0           ['conv2d_169[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_170 (Conv2D)            (None, 8, 8, 256)    295168      ['max_pooling2d_68[0][0]']       
                                                                                                  
 dropout_101 (Dropout)          (None, 8, 8, 256)    0           ['conv2d_170[0][0]']             
                                                                                                  
 conv2d_171 (Conv2D)            (None, 8, 8, 256)    590080      ['dropout_101[0][0]']            
          

In [20]:
from tensorflow.keras import backend as K

def custom_attention(inputs, reduction_ratio=16, num_groups=8):
    # Efficient Channel Attention (ECA)
    channels = inputs.shape[-1]
    eca_avg = K.mean(inputs, axis=(1, 2))
    eca_attention = tf.keras.layers.Conv1D(1, kernel_size=1, activation='sigmoid')(eca_avg)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    eca_attention = K.expand_dims(eca_attention, axis=1)
    x = inputs * eca_attention

    # Coordinate Attention (CA)
    ca_channel_avg = K.mean(x, axis=2, keepdims=True)
    ca_channel_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(ca_channel_avg)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    ca_channel_attention = K.expand_dims(ca_channel_attention, axis=2)
    x = x * ca_channel_attention

    # Adaptive Attention
    adaptive_avg = K.mean(x, axis=(1, 2))
    adaptive_attention = tf.keras.layers.Dense(1, activation='sigmoid')(adaptive_avg)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    x = x * adaptive_attention

    return x

def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh((np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

def coarse_net():
    # Input layer
    inputs = tf.keras.layers.Input((128, 128, 1))

    # Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.BatchNormalization()(c1)  # Add BatchNormalization
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.BatchNormalization()(c2)  # Add BatchNormalization
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.BatchNormalization()(c3)  # Add BatchNormalization
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.BatchNormalization()(c4)  # Add BatchNormalization
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    u6 = custom_attention(u6)  # Apply custom attention
    u6 = tf.keras.layers.GroupNormalization()(u6)  # Add BatchNormalization
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    u7 = custom_attention(u7)  # Apply custom attention
    u7 = tf.keras.layers.GroupNormalization()(u7)  # Add BatchNormalization
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    u8 = custom_attention(u8)  # Apply custom attention
    u8 = tf.keras.layers.GroupNormalization()(u8)  # Add BatchNormalization
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=gelu, kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    
    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    u9_attended = custom_attention(u9)
    u9 = tf.keras.layers.GroupNormalization()(u9)  # Add BatchNormalization
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9_attended)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    # Create the model
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_162 (Conv2D)            (None, 128, 128, 16  160         ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 dropout_97 (Dropout)           (None, 128, 128, 16  0           ['conv2d_162[0][0]']             
                                )                                                             

In [ ]:

# Adaptive Mixed Domain attention
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_attention(inputs, reduction_ratio=16, num_groups=8):
    # Adaptive Attention
    adaptive_avg = K.mean(inputs, axis=(1, 2))
    adaptive_attention = tf.keras.layers.Dense(1, activation='sigmoid')(adaptive_avg)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    adaptive_attention = K.expand_dims(adaptive_attention, axis=1)
    x = inputs * adaptive_attention

    # Lightweight Mixed-Domain Attention
    domain_avg = K.mean(x, axis=(1, 2))
    domain_attention = tf.keras.layers.Dense(1, activation='sigmoid')(domain_avg)
    domain_attention = K.expand_dims(domain_attention, axis=1)
    domain_attention = K.expand_dims(domain_attention, axis=1)
    x = x * domain_attention

    # CBAM (Channel and Spatial Attention)
    channels = x.shape[-1]
    channel_avg = K.mean(x, axis=(1, 2))
    channel_attention = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(channel_avg)
    channel_attention = tf.keras.layers.Dense(channels, activation='sigmoid')(channel_attention)
    channel_attention = K.expand_dims(channel_attention, axis=1)
    channel_attention = K.expand_dims(channel_attention, axis=1)
    x = x * channel_attention
    
    # Convolutional Triplet Attention Module
    conv_triplet_attention = tf.keras.layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)
    x = x * conv_triplet_attention

    return x

In [ ]:
import torch
from torch_sparse import SparseTensor
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops
from torch_geometric.data import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
import matplotlib.pyplot as plt

# Create a sample data object
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

# Define GAT model
class SparseGATLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SparseGATLayer, self).__init__('add')
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.att = torch.nn.Linear(2 * out_channels, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x, edge_index):
        x = self.lin(x)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        edge_weight = torch.ones(edge_index.size(1))  # Uniform edge weights
        edge_index = SparseTensor(row=edge_index[0], col=edge_index[1],
                                  value=edge_weight, sparse_sizes=(x.size(0), x.size(0)))
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        return x_j

    def update(self, aggr_out, x):
        alpha = self.att(torch.cat([aggr_out, x], dim=1))
        alpha = torch.softmax(alpha, dim=1)
        x = alpha * x
        return self.relu(x)

class GATNet(torch.nn.Module):
    def __init__(self):
        super(GATNet, self).__init__()
        self.conv1 = SparseGATLayer(3, 128)
        self.conv2 = SparseGATLayer(128, 64)
        self.conv3 = SparseGATLayer(64, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        return x

# Create GAT network
model = GATNet()

# Convert data to PyTorch DataLoader
loader = DataLoader([data], batch_size=1)

# Process data using GAT network
output = model(data)

# Plot input image
plt.subplot(1, 2, 1)
plt.imshow(data.x.numpy())
plt.title('Input Image')

# Plot output image
plt.subplot(1, 2, 2)
plt.imshow(output.detach().numpy())
plt.title('Output Image')

plt.show()


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops
from torch_geometric.data import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
import matplotlib.pyplot as plt

# Create a sample data object
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

# Define GAT model
class GATLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GATLayer, self).__init__('add')
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.att = torch.nn.Linear(2 * out_channels, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x, edge_index):
        x = self.lin(x)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        return self.propagate(edge_index, x=x)

    def message(self, x_j, edge_index_i):
        return x_j

    def update(self, aggr_out, x):
        alpha = self.att(torch.cat([aggr_out, x], dim=1))
        alpha = torch.softmax(alpha, dim=1)
        x = torch.squeeze(torch.matmul(torch.unsqueeze(alpha, 0), x))
        return self.relu(x)

class GATNet(torch.nn.Module):
    def __init__(self):
        super(GATNet, self).__init__()
        self.conv1 = GATLayer(3, 128)
        self.conv2 = GATLayer(128, 64)
        self.conv3 = GATLayer(64, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        return x

# Create GAT network
model = GATNet()

# Convert data to PyTorch DataLoader
loader = DataLoader([data], batch_size=1)

# Process data using GAT network
output = model(data)

# Plot input image
plt.subplot(1, 2, 1)
plt.imshow(data.x.numpy())
plt.title('Input Image')

# Plot output image
plt.subplot(1, 2, 2)
plt.imshow(output.detach().numpy())
plt.title('Output Image')

plt.show()



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class SparseGAT(layers.Layer):
    def __init__(self, output_dim):
        super(SparseGAT, self).__init__()
        self.output_dim = output_dim
        self.fc = layers.Dense(output_dim)

    def call(self, inputs, training=False):
        x, edge_index = inputs
        x = self.fc(x)
        x = tf.nn.leaky_relu(x, alpha=0.2)

        # Sparse GAT computation
        row, col = edge_index[0], edge_index[1]
        edge_weight = tf.ones((edge_index.shape[1],), dtype=tf.float32)
        edge_weight = tf.SparseTensor(indices=tf.transpose(edge_index),
                                      values=edge_weight,
                                      dense_shape=(x.shape[0], x.shape[0]))
        x = tf.sparse.sparse_dense_matmul(edge_weight, x)

        # Channel Attention (CA)
        ca = tf.reduce_mean(x, axis=-1, keepdims=True)  # Compute channel-wise mean
        ca = self.fc(ca)
        ca = tf.nn.sigmoid(ca)
        x_ca = tf.multiply(x, ca)  # Apply channel-wise attention

        # Spatial Attention (SA)
        sa = tf.reduce_mean(x_ca, axis=-1, keepdims=True)  # Compute spatial-wise mean
        sa = self.fc(sa)
        sa = tf.nn.sigmoid(sa)
        x_sa = tf.multiply(x_ca, sa)  # Apply spatial-wise attention

        # Patch Attention (PA)
        patch_size = 16  # Define the patch size (adjust as needed)
        x_reshaped = tf.reshape(x_sa, (-1, patch_size, patch_size, self.output_dim))  # Reshape to patches
        pa = self.fc(x_reshaped)
        pa = tf.nn.sigmoid(pa)
        x_pa = tf.multiply(x_reshaped, pa)  # Apply patch-wise attention
        x_pa = tf.reshape(x_pa, (-1, x_pa.shape[1] * x_pa.shape[2], self.output_dim))  # Reshape back to original shape

        # Fusion of attention
        x_fusion = tf.reduce_sum(x_pa, axis=-1)

        return x_fusion
